In [51]:
import sys
import os

# Add the src/ folder to Python's import path
src_path = os.path.abspath(os.path.join("..", "src"))
if src_path not in sys.path:
    sys.path.append(src_path)

In [52]:
from features import (
    compute_inter_iris_distance,
    compute_average_diameter,
    normalise_distance,
    compute_time_series_std,
    flag_mTBI
)

In [53]:
import pandas as pd

df = pd.read_csv("/Users/nabs/iris-mtbi-screening/data/raw/control_sample.csv")
df.head()

,left_mid_x,left_mid_y,left_inner_x,left_inner_y,left_outer_x,left_outer_y,right_mid_x,right_mid_y,right_inner_x,right_inner_y,right_outer_x,right_outer_y
0,99.929206,100.111285,94.853328,100.111285,104.979520,100.111285,150.118846,99.926424,145.050594,99.926424,155.191678,99.926424
1,100.049920,99.995039,95.014171,99.995039,105.031630,99.995039,150.033877,99.938508,144.803541,99.938508,154.882406,99.938508
2,99.874280,99.762899,94.937081,99.762899,104.973596,99.762899,149.815803,100.144929,144.937602,100.144929,154.781077,100.144929
3,100.078165,100.040292,95.057908,100.040292,105.041529,100.040292,149.913690,100.111574,144.848777,100.111574,154.884736,100.111574
4,100.119091,100.124196,95.151338,100.124196,105.204822,100.124196,150.123928,99.969910,145.097228,99.969910,155.100962,99.969910


In [54]:
norm_distances = []

for i, row in df.iterrows():
    left_mid = (row["left_mid_x"], row["left_mid_y"])
    right_mid = (row["right_mid_x"], row["right_mid_y"])

    left_inner = (row["left_inner_x"], row["left_inner_y"])
    left_outer = (row["left_outer_x"], row["left_outer_y"])
    right_inner = (row["right_inner_x"], row["right_inner_y"])
    right_outer = (row["right_outer_x"], row["right_outer_y"])

    inter_iris = compute_inter_iris_distance(left_mid, right_mid)
    avg_diameter = compute_average_diameter(left_inner, left_outer, right_inner, right_outer)
    norm = normalise_distance(inter_iris, avg_diameter)

    norm_distances.append(norm)

# Remove any invalid values
norm_distances = [x for x in norm_distances if pd.notna(x)]

In [55]:
std_dev = compute_time_series_std(norm_distances)
is_mtbi = flag_mTBI(std_dev)

print(f"Standard deviation: {std_dev:.4f}")
print("⚠️  Likely mTBI" if is_mtbi else "✅ No mTBI detected")

Standard deviation: 0.0525
✅ No mTBI detected
